In [6]:
## Setup
from notebook_utils import resolve_paths_from_parent_directory
# auto reload notebook deps
%reload_ext autoreload
%autoreload 2
resolve_paths_from_parent_directory()

In [11]:
import pandas as pd

def custom_feature_set_without_trip_id(dataset):
    train, test = dataset
    train = pd.get_dummies(train, columns=['route', 'trip_id'])

    # compute average passenger_count by next_stop_id (using training only so as not to bake in information about the test set)
    train_stop_stats = train[
        ['next_stop_id', 'passenger_count']
    ].groupby('next_stop_id').agg({'passenger_count':['mean', 'std']})
    train['avg_stop_passengers'] = train['next_stop_id'].apply(lambda x: train_stop_stats[('passenger_count', 'mean')].loc[x])
    test['avg_stop_passengers'] = test['next_stop_id'].apply(lambda x: train_stop_stats[('passenger_count', 'mean')].loc[x])

    test = pd.get_dummies(test, columns=['route'])

    non_features =  ['service_date', 'vehicle_id', 'timestamp', 'prior_stop_id', 'next_stop_id']

    return train.drop(columns=non_features), test.drop(columns=non_features)

In [12]:
from data_loader import load_global_feature_set
from run_experiment import run_experiment
from feature_sets import feature_set_without_trip_id
from xgboost import XGBRegressor
from sklearn.linear_model import LassoCV

data_dir = "../../data"
route_str = "B46"

## Prepare globlal feature set
df_route, stop_dict = load_global_feature_set(data_dir, route_str)

for model in [LassoCV(), XGBRegressor()]:
    print(f"Running experiment for {model}...")
    experiment_eval = run_experiment(
        df_route,
        custom_feature_set_without_trip_id,
        model,
        stop_dict
    )

    print("Evaluation")
    print(experiment_eval.basic_eval('train'))


Running experiment for LassoCV()...
Selecting features...


NameError: name 'pd' is not defined